In [3]:
import torch

# Parameters
volume_shape = (128, 128, 128)
step_size = 0.1
max_steps = int(volume_shape[0] / step_size)
max_steps

1280

In [4]:
x = torch.linspace(-1, 1, volume_shape[0])
y = torch.linspace(-1, 1, volume_shape[1])
z = torch.linspace(-1, 1, volume_shape[2])

In [6]:
x, y, z = torch.meshgrid(x, y, z)

/Users/drwho/anaconda3/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [16]:
(x[:,0]==x[:,1]).float().min(), (x[:,:,0]==x[:,:,1]).float().min()

(tensor(1.), tensor(1.))

In [18]:
(y[0]==y[1]).float().min(), (y[:,:,0]==y[:,:,1]).float().min()

(tensor(1.), tensor(1.))

In [19]:
(z[0]==z[1]).float().min(), (z[:,0]==z[:,1]).float().min()

(tensor(1.), tensor(1.))

In [21]:
volume = torch.exp(-(x**2 + y**2 + z**2))

In [22]:
def ray_march(volume, ray_origin, ray_direction, step_size, max_steps):
    accumulated_color = torch.zeros(3)
    accumulated_opacity = 0.0
    position = ray_origin.clone()
    
    for _ in range(max_steps):
        # Sample the volume at the current position
        i, j, k = (position * torch.tensor(volume.shape)).long()
        if 0 <= i < volume.shape[0] and 0 <= j < volume.shape[1] and 0 <= k < volume.shape[2]:
            density = volume[i, j, k]
            color = torch.tensor([1.0, 0.5, 0.0])  # Assume orange color for simplicity
            opacity = density * step_size  # Compute opacity based on density
            
            # Accumulate color and opacity
            accumulated_color += (1 - accumulated_opacity) * color * opacity
            accumulated_opacity += opacity
        
        # Move to the next position along the ray
        position += ray_direction * step_size
        
        # Early exit if opacity is close to 1
        if accumulated_opacity >= 1.0:
            break
    
    return accumulated_color